# Word patterns

## "Big words"

We want to count the words in the BHSA, but not the words as members of the `word` type, but the words in as far as they
are written together.

In [1]:
from tf.app import use

In [2]:
A = use("ETCBC/bhsa", hoist=globals())

**Locating corpus resources ...**

Name,# of nodes,# slots/node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


We start by putting all words (in the sense of contiguous stretches of letters without white space)
together in one list.

In order to do that, we'll use the `trailer` feature on words.

Let's first inspect that feature by means of a frequency list.

In [3]:
F.trailer.freqList()

((' ', 236930),
 ('', 121801),
 ('&', 42275),
 ('00 ', 20146),
 ('05 ', 2266),
 ('00_S ', 1892),
 ('00_P ', 1165),
 ('_S ', 76),
 (' 05 ', 17),
 ('_P ', 13),
 ('00_N ', 7),
 ('00_N_P ', 1),
 ('00_N_S ', 1))

The `''` value is used when the word is glued to the next word. We take this as a non-word separator.

The `'&'` is used to indicate a maqqef. We take this as a word separator.
All other values are punctuation marks that imply word separation.

So, we make a list of all words where we glue words together if their trailer is the empty string.

We accumulate the words in a list `words`.
The items in this list are lists of word nodes, the nodes that belong to words that are separated by the
empty string.

For each word node that we visit, we examine the trailer of the last node of the last entry.
If it is the empty string, we append the node to that entry.
Otherwise we make a new entry.

In [4]:
words = []

for w in F.otype.s("word"):
    if len(words) == 0:
        words.append([w])
        continue
        
    lastEntry = words[-1]
    lastNode = lastEntry[-1]
    lastTrailer = F.trailer.v(lastNode)
    if lastTrailer == "":
        lastEntry.append(w)
    else:
        words.append([w])

Let's see whether we have done this right.

We show the first entries of our word list:

In [5]:
for entry in words[0:11]:
    print(entry)

[1, 2]
[3]
[4]
[5]
[6, 7]
[8, 9]
[10, 11]
[12, 13, 14]
[15]
[16]
[17, 18]


This seems to go well, but let's print out the words themselves:

In [6]:
for entry in words[0:11]:
    A.plainTuple(entry[::-1])

Yes, the entries correspond to the words in the new sense.

So, how many do we have?

In [7]:
len(words)

304789

## Final consonants

What is the frequency of final consonants w.r.t. their non final counterparts?

In [8]:
import re
import collections

First a little investment in naming the consonants and linking the final ones to the non-final ones.

In [9]:
# keys: final consonants
# values: corresponding non-final consonants

theFive = dict(
    kafF="\u05da",
    kaf="\u05db",
    memF="\u05dd",
    mem="\u05de",
    nunF="\u05df",
    nun="\u05e0",
    peF="\u05e3",
    pe="\u05e4",
    tsadiF="\u05e5",
    tsadi="\u05e6",
)

for (name, ch) in theFive.items():
    print(f"{name} = {ch}")

kafF = ך
kaf = כ
memF = ם
mem = מ
nunF = ן
nun = נ
peF = ף
pe = פ
tsadiF = ץ
tsadi = צ


Here comes the linking.

In [10]:
finalConsonants = {}

for name in theFive:
    if name.endswith("F"):
        finalConsonants[theFive[name]] = theFive[name[0:-1]]
        
for (cF, c) in finalConsonants.items():
    print(f"final {cF} corresponds to {c}")

final ך corresponds to כ
final ם corresponds to מ
final ן corresponds to נ
final ף corresponds to פ
final ץ corresponds to צ


Now we construct a regular expression that matches all of these consonants.

In [11]:
finals = "".join(finalConsonants.keys())
nonFinals = "".join(finalConsonants.values())
theFiveRe = re.compile(fr"([{finals}{nonFinals}])")
theFiveRe

re.compile(r'([ךםןףץכמנפצ])', re.UNICODE)

We are going to match each of the 400,000+ words to this pattern, and collect all matches of it.

In [12]:
freqs = collections.Counter()

for w in F.otype.s("word"):
    text  = F.g_word_utf8.v(w)
    
    chars = theFiveRe.findall(text)
    for ch in chars:
        freqs[ch] += 1

We compute the proportion in percents of consonants against their counterparts,
we also give totals.

We present the result as a markdown table.

In [13]:
from tf.advanced.helpers import dm

In [14]:
text = """
consonant | #total | #final | #nonfinal | %final
--- | --- | --- | --- | ---
"""

for (consF, cons) in sorted(finalConsonants.items()):
    final = freqs[consF]
    nonFinal = freqs[cons]
    total = final + nonFinal
    percent = int(round(final * 100 / total))
    text += f"{cons} and {consF} | {total} | {final} | {nonFinal} | {percent}\n"
    
dm(text)


consonant | #total | #final | #nonfinal | %final
--- | --- | --- | --- | ---
כ and ך | 47469 | 14002 | 33467 | 29
מ and ם | 98929 | 41291 | 57638 | 42
נ and ן | 55093 | 15245 | 39848 | 28
פ and ף | 18284 | 2554 | 15730 | 14
צ and ץ | 14977 | 3288 | 11689 | 22


# Remaining questions

## Have we seen all the consonants?

The text of the BHSA is mainly store in the feature
[`g_word_utf8`](https://etcbc.github.io/bhsa/features/g_word_utf8/),
but there are other features as well that store material:

* qere material that deviates from the ketiv is in
  [`qere_utf8`](https://etcbc.github.io/bhsa/features/qere_utf8/)
  
* [`trailer_utf8`](https://etcbc.github.io/bhsa/features/trailer_utf8/) stores inter-word material, but can contain
  consonants. Again, if there is a deviating qere, the inter-word material is in 
  [`qere_trailer_utf8`](https://etcbc.github.io/bhsa/features/qere_trailer_utf8/).
  
These factors may cause discrepancies in different ways of counting consonants.


# Exploration

Which consonants occur in `trailer_utf8` and `qere_trailer_utf8`?

In [15]:
nonFinalConsonants = set(finalConsonants.values())

for (chars, n) in sorted(F.trailer_utf8.freqList()):
    print(f"{n:>6}x {repr(chars)}")
    for c in chars:
        if c in finalConsonants:
            extra = "final consonant"
        elif c in nonFinalConsonants:
            extra = "non-final consonant"
        else:
            extra = ""
        if extra:
            print(f"\t{extra:<20} {c}")

121801x ''
236930x ' '
    17x ' ׀ '
    76x ' ס '
    13x ' פ '
	non-final consonant  פ
 42275x '־'
  2266x '׀ '
 20146x '׃ '
     7x '׃ ׆ '
     1x '׃ נ ס '
	non-final consonant  נ
     1x '׃ נ פ '
	non-final consonant  נ
	non-final consonant  פ
  1892x '׃ ס '
  1165x '׃ פ '
	non-final consonant  פ


OK, there are consonants, and they include some consonants we are interested in,
the pe and the nun, both non-final.

But they they are really the petuhah and the nun-hafukha and they do not function as consonants.
See [writing](https://annotation.github.io/text-fabric/tf/writing/hebrew.html).

# Ketiv/qere sensitive counting

So let's add a layer of sophistication, and redo the counting in several ways:

* Q+ the qere text, with interword material
* Q- the qere text, without interword material
* K+ the ketiv text, with interword material
* K- the ketiv text, without interword material

In [16]:
freqs = collections.defaultdict(lambda: collections.Counter())

for w in F.otype.s("word"):
    ketiv = F.g_word_utf8.v(w)
    qere = F.qere_utf8.v(w) or ketiv
    trailerk = F.trailer_utf8.v(w)
    trailerq = F.qere_trailer_utf8.v(w) or trailerk
    
    texts = {}
    texts["Km"]  = ketiv
    texts["Qm"]  = qere
    texts["Kp"] = f"{ketiv}{trailerk}"
    texts["Qp"] = f"{qere}{trailerq}"
    
    for (kind, text) in texts.items():
        chars = theFiveRe.findall(text)
        for ch in chars:
            freqs[kind][ch] += 1

Now we have to display the results:

In [17]:
text = """
consonant | kind | #totalQp | #final | #nonfinal | %final
--- | --- | --- | --- | ---| ---
"""

for (consF, cons) in sorted(finalConsonants.items()):
    for kind in ("Qp", "Qm", "Kp", "Km"):
        theseFreqs = freqs[kind]
        
        final = theseFreqs[consF]
        nonFinal = theseFreqs[cons]
        total = final + nonFinal
        percent = int(round(final * 100 / total))
        text += f"{cons} and {consF} | {kind} | {total} | {final} | {nonFinal} | {percent}\n"
    
dm(text)


consonant | kind | #totalQp | #final | #nonfinal | %final
--- | --- | --- | --- | ---| ---
כ and ך | Qp | 47482 | 14001 | 33481 | 29
כ and ך | Qm | 47482 | 14001 | 33481 | 29
כ and ך | Kp | 47469 | 14002 | 33467 | 29
כ and ך | Km | 47469 | 14002 | 33467 | 29
מ and ם | Qp | 98940 | 41291 | 57649 | 42
מ and ם | Qm | 98940 | 41291 | 57649 | 42
מ and ם | Kp | 98929 | 41291 | 57638 | 42
מ and ם | Km | 98929 | 41291 | 57638 | 42
נ and ן | Qp | 55093 | 15240 | 39853 | 28
נ and ן | Qm | 55091 | 15240 | 39851 | 28
נ and ן | Kp | 55095 | 15245 | 39850 | 28
נ and ן | Km | 55093 | 15245 | 39848 | 28
פ and ף | Qp | 19457 | 2555 | 16902 | 13
פ and ף | Qm | 18278 | 2555 | 15723 | 14
פ and ף | Kp | 19463 | 2554 | 16909 | 13
פ and ף | Km | 18284 | 2554 | 15730 | 14
צ and ץ | Qp | 14977 | 3288 | 11689 | 22
צ and ץ | Qm | 14977 | 3288 | 11689 | 22
צ and ץ | Kp | 14977 | 3288 | 11689 | 22
צ and ץ | Km | 14977 | 3288 | 11689 | 22


## Are there words with multiple final consonants?

An earlier version of this notebook counted at most 1 final consonant per word and that gave
slight discrepancies with the result above. Let's first reconstruct that way of counting and list the results.

We only do the `Qm` variety: we follow the ketiv, and do not look at trailing material.

In [18]:
freqs = collections.defaultdict(lambda: collections.Counter())
finalsRe = re.compile(fr"([{finals}])")

for w in F.otype.s("word"):
    ketiv = F.g_word_utf8.v(w)
    
    # new method
    chars = theFiveRe.findall(ketiv)
    for ch in chars:
        freqs["new"][ch] += 1
        
    # old method for finding final consonants
    match = finalsRe.search(ketiv)
    if match:
        consF = match.group(1)
        freqs["old"][consF] += 1
    

In [19]:
text = """
consonant | kind | #totalQp | #final | #nonfinal | %final
--- | --- | --- | --- | ---| ---
"""

for (consF, cons) in sorted(finalConsonants.items()):
    for kind in ("new", "old"):
        theseFreqs = freqs[kind]
        
        final = theseFreqs[consF]
        if kind == "new":
            finalNew = final
            finalRep = final
            nonFinal = theseFreqs[cons]
            total = final + nonFinal
            percent = int(round(final * 100 / total))
            rep = f"{cons} and {consF}"
        else:
            diff = final - finalNew
            sign = "+" if diff > 0 else "-" if diff < 0 else ""
            diff = -diff if diff < 0 else diff
            finalRep = f"{final} ({sign}{diff})"
            nonFinal = ""
            total = ""
            percent = ""
            rep = consF
        text += f"{rep} | {kind} | {total} | {finalRep} | {nonFinal} | {percent}\n"
    
dm(text)


consonant | kind | #totalQp | #final | #nonfinal | %final
--- | --- | --- | --- | ---| ---
כ and ך | new | 47469 | 14002 | 33467 | 29
ך | old |  | 13998 (-4) |  | 
מ and ם | new | 98929 | 41291 | 57638 | 42
ם | old |  | 41274 (-17) |  | 
נ and ן | new | 55093 | 15245 | 39848 | 28
ן | old |  | 15236 (-9) |  | 
פ and ף | new | 18284 | 2554 | 15730 | 14
ף | old |  | 2554 (0) |  | 
צ and ץ | new | 14977 | 3288 | 11689 | 22
ץ | old |  | 3286 (-2) |  | 


So, yes indeed, there must be words with multiple final consonants.

Let's find and show them!

In [20]:
multiFinal = collections.defaultdict(list)

for w in F.otype.s("word"):
    ketiv = F.g_word_utf8.v(w)
    chars = finalsRe.findall(ketiv)
    if len(chars) > 1:
        multiFinal[tuple(chars)].append(w)

multiFinal

defaultdict(list,
            {('ן', 'ץ'): [90929, 90933],
             ('ן', 'ן'): [120322,
              121905,
              145492,
              384040,
              389715,
              393118,
              393275],
             ('ן', 'ם'): [121552,
              121924,
              123233,
              123677,
              124944,
              128871,
              128880,
              128927,
              128934,
              238858,
              238891,
              244151,
              244276,
              252286,
              289899,
              420374,
              423988],
             ('ך', 'ן'): [208444, 225386],
             ('ן', 'ך'): [210146, 346890],
             ('ך', 'ך'): [262783],
             ('ם', 'ך'): [306728]})

Tantalizing. Let's show it in a prettier way:

In [21]:
for (chars, words) in sorted(multiFinal.items()):
    nWords = len(words)
    plural = "" if nWords == 1 else "s"
    dm(f"### <span class='txtu hbo'>{' '.join(chars)}</span>\n\n")
    dm(f"In {nWords} word{plural}:\n\n")
    for w in words:
        A.pretty(w, withNodes=True, fmt="text-orig-full-ketiv")

### <span class='txtu hbo'>ך ך</span>



In 1 word:



### <span class='txtu hbo'>ך ן</span>



In 2 words:



### <span class='txtu hbo'>ם ך</span>



...

In 17 words:



### <span class='txtu hbo'>ן ן</span>



In 7 words:



### <span class='txtu hbo'>ן ץ</span>



In 2 words:



Mystery solved! There are things that are single words in the BHSA and yet have spaces or maqqefs in them.

# "Big words" revisited

Let's redo the count of the *big words* where we will split single words that have a maqqef or space in them.

We also collect the words with a space or maqqef in them.

In [26]:
words = []
wordsSpace = set()
wordsMaqqef = set()

MAQ = "\u05be"

wordSplitRe = re.compile(rf" |{MAQ}")

for w in F.otype.s("word"):
    ketiv = F.g_word_utf8.v(w)
    chunks = wordSplitRe.split(ketiv)
    if MAQ in ketiv:
        wordsMaqqef.add(w)
    if " " in ketiv:
        wordsSpace.add(w)
    
    start = 0
    if len(words) == 0:
        for chunk in chunks:
            words.append([(w, start, chunk)])
            start += len(chunk) + 1
        continue

    lastEntry = words[-1]
    lastNode = lastEntry[-1][0]
    lastTrailer = F.trailer.v(lastNode)

    chunk = chunks[0]
    start = 0
    if lastTrailer == "":
        lastEntry.append((w, start, chunk))
    else:
        words.append([(w, start, chunk)])

    start += len(chunk)
    for chunk in chunks[1:]:
        words.append([(w, start, chunk)])
        start += len(chunk) + 1

Let's see whether we have done this right.

We show the first entries of our word list:

In [27]:
for entry in words[0:11]:
    print(entry)

[(1, 0, 'בְּ'), (2, 0, 'רֵאשִׁ֖ית')]
[(3, 0, 'בָּרָ֣א')]
[(4, 0, 'אֱלֹהִ֑ים')]
[(5, 0, 'אֵ֥ת')]
[(6, 0, 'הַ'), (7, 0, 'שָּׁמַ֖יִם')]
[(8, 0, 'וְ'), (9, 0, 'אֵ֥ת')]
[(10, 0, 'הָ'), (11, 0, 'אָֽרֶץ')]
[(12, 0, 'וְ'), (13, 0, 'הָ'), (14, 0, 'אָ֗רֶץ')]
[(15, 0, 'הָיְתָ֥ה')]
[(16, 0, 'תֹ֨הוּ֙')]
[(17, 0, 'וָ'), (18, 0, 'בֹ֔הוּ')]


This seems to go well, but let's see a case where the word has a maqqef or a space.

In [ ]:
sorted(wordsSpace | wordsMaqqef)

In [47]:
totalMaqqef = 0
totalSpace = 0

for entry in words:
    withMaqqef = False
    withSpace = False
    for chunk in entry:
        node = chunk[0]
        if node in wordsMaqqef:
            withMaqqef = True
        if node in wordsSpace:
            withSpace = True
    if withMaqqef and totalMaqqef < 10:
        totalMaqqef += 1
        print(entry)
    if withSpace and totalSpace < 10:
        totalSpace += 1
        print(entry)
    if totalSpace >= 10 and totalMaqqef >= 10:
        break

[(2033, 0, 'תּ֣וּבַל')]
[(2033, 8, 'קַ֔יִן')]
[(2042, 0, 'תּֽוּבַל')]
[(2042, 8, 'קַ֖יִן')]
[(4687, 0, 'רְחֹבֹ֥ת')]
[(4687, 8, 'עִ֖יר')]
[(5641, 0, 'לְ'), (5642, 0, 'בֵֽית')]
[(5642, 5, 'אֵ֖ל')]
[(5646, 0, 'בֵּֽית')]
[(5646, 6, 'אֵ֤ל')]
[(5905, 0, 'בֵּֽית')]
[(5905, 6, 'אֵ֑ל')]
[(5917, 0, 'בֵּֽית')]
[(5917, 6, 'אֵ֖ל')]
[(6290, 0, 'בְּ'), (6291, 0, 'עַשְׁתְּרֹ֣ת')]
[(6291, 12, 'קַרְנַ֔יִם')]
[(6313, 0, 'אֵ֣יל')]
[(6313, 5, 'פָּארָ֔ן')]
[(6323, 0, 'עֵ֤ין')]
[(6323, 5, 'מִשְׁפָּט֙')]


Observe the words 5641 and 5642:

In [49]:
A.pretty(5641)
A.pretty(5642)

The BHSA splits them as

* `(L)`
* `(BT-AL)`

But the common-sense split will be

* `(LBT)`
* `(AL)`

Anyway, how many do we have in total now?

In [50]:
len(words)

305506